# Selective flow constraints with [PowerSimulations.jl](https://github.com/NREL/PowerSimulations.jl)

**Originally Contributed by**: Clayton Barrows

## Introduction

This is a basic simulation example using the [TAMU Cases](https://electricgrids.engr.tamu.edu/).

## Dependencies

In [1]:
using SIIPExamples
pkgpath = dirname(dirname(pathof(SIIPExamples)))
using Dates

### Modeling Packages

In [2]:
using PowerSystems
using PowerSimulations

### Optimization packages
For this simple example, we can use the Cbc solver with a relatively relaxed tolerance.

In [3]:
using Cbc #solver
solver = optimizer_with_attributes(Cbc.Optimizer, "logLevel" => 1, "ratioGap" => 0.5)

MathOptInterface.OptimizerWithAttributes(Cbc.Optimizer, Pair{MathOptInterface.AbstractOptimizerAttribute,Any}[MathOptInterface.RawParameter("logLevel") => 1, MathOptInterface.RawParameter("ratioGap") => 0.5])

### Create a `System` from TAMU data
We have included some of the TAMU cases (with truncated time series info)
in the [PowerSystemsTestData](https://github.com/nrel-siip/powersystemstestdata)
repository for testing, so we can just use that.

In [4]:
PowerSystems.download(PowerSystems.TestData; branch = "master") # *note* add `force=true` to get a fresh copy
base_dir = dirname(dirname(pathof(PowerSystems)));

The TAMU data format relies on a folder containing `.m` or `.raw` files and `.csv`
files for the time series data. We have provided a parser for the TAMU data format with
the `TamuSystem()` fuction.

In [5]:
TAMU_DIR = joinpath(base_dir, "data", "ACTIVSg2000");
sys = TamuSystem(TAMU_DIR)
horizon = 24;  interval = Dates.Hour(24);
transform_single_time_series!(sys, horizon, interval);

┌ Info: The PSS(R)E parser currently supports buses, loads, shunts, generators, branches, transformers, and dc lines
└ @ PowerSystems /Users/sdalvi/.julia/packages/PowerSystems/eF3Pv/src/parsers/pm_io/common.jl:25
┌ Info: Switched shunt converted to fixed shunt, with default value gs=0.0
└ @ PowerSystems /Users/sdalvi/.julia/packages/PowerSystems/eF3Pv/src/parsers/pm_io/psse.jl:312
┌ Info: reversing the orientation of branch 36 (1071, 1019) to be consistent with other parallel branches
└ @ PowerSystems /Users/sdalvi/.julia/packages/PowerSystems/eF3Pv/src/parsers/pm_io/data.jl:1216
┌ Info: no active generators found at bus 6144, updating to bus type from 2 to 1
└ @ PowerSystems /Users/sdalvi/.julia/packages/PowerSystems/eF3Pv/src/parsers/pm_io/data.jl:1574
┌ Info: no active generators found at bus 8070, updating to bus type from 2 to 1
└ @ PowerSystems /Users/sdalvi/.julia/packages/PowerSystems/eF3Pv/src/parsers/pm_io/data.jl:1574
┌ Info: no active generators found at bus 6273, updating

## Run a PCM
note that the TAMU data doesn't contain startup and shutdown costs, or minimum up/down
time limits, so a UC problem merely respects minmum generation levels.

In [6]:
sim_folder = mkpath(joinpath(pkgpath, "TAMU-sim"))
stages_definition = Dict(
    "UC" =>
        Stage(
            GenericOpProblem,
            template_unit_commitment(network = CopperPlatePowerModel),
            sys,
            solver;),
)
order = Dict(1 => "UC")
horizons = Dict("UC" => 24)
intervals = Dict("UC" => (Hour(24), Consecutive()))
DA_sequence = SimulationSequence(
    step_resolution = Hour(24),
    order = order,
    horizons = horizons,
    intervals = intervals,
    ini_cond_chronology = IntraStageChronology(),
)

┌ Warning: No Feedforward Chronologies have been defined. This configuration assummes that there is no information passing between stages
└ @ PowerSimulations /Users/sdalvi/.julia/packages/PowerSimulations/0nHyl/src/core/simulation_sequence.jl:126


Feed Forward Chronology
-----------------------

Initial Condition Chronology
----------------------------

1



### Define and build a simulation

In [7]:
sim = Simulation(
    name = "TAMU-test",
    steps = 3,
    stages = stages_definition,
    stages_sequence = DA_sequence,
    simulation_folder = sim_folder,
)

build!(sim)

### Execute the simulation

In [8]:
sim_results = execute!(sim)

Executing Step 1
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Nov  9 2020 

command line - Cbc_C_Interface -ratioGap 0.5 -logLevel 1 -solve -quit (default strategy 1)
ratioGap was changed from 0 to 0.5
Continuous objective value is 1.06275e+06 - 0.84 seconds
Cgl0004I processed model has 12849 rows, 26378 columns (5314 integer (5314 of which binary)) and 61654 elements
Cbc0045I Trying just fixing integer variables (and fixingish SOS).
Cbc0045I Warning: mipstart values could not be used to build a solution.
Cbc0012I Integer solution of 1087676.9 found by DiveCoefficient after 0 iterations and 0 nodes (4.29 seconds)
Cbc0011I Exiting as integer gap of 24929.285 less than 1e-10 or 50%
Cbc0001I Search completed - best objective 1087676.906684934, took 0 iterations and 0 nodes (4.31 seconds)
Cbc0035I Maximum depth 0, 0 variables fixed on reduced cost
Cuts at root node changed objective from 1.06275e+06 to 1.06275e+06
Probing was tried 0 times and created 0 cuts of which 0 were

Simulation Results Reference

Results Folder: /Users/sdalvi/Documents/GitRepos/SIIPExamples.jl/TAMU-sim/TAMU-test/15/results

Reference Tables

________________

stage-UC

start__ThermalStandard: dataframe size (3, 3)

P__ThermalStandard: dataframe size (3, 3)

stop__ThermalStandard: dataframe size (3, 3)

parameter_P_PowerLoad: dataframe size (3, 3)

On__ThermalStandard: dataframe size (3, 3)

stage-UC

time length: 24

### Load and analyze results

In [9]:
uc_results = load_simulation_results(sim_results, "UC");

uc_results.variable_values[:On__ThermalStandard]

,generator-7135-1,generator-7196-1,generator-2076-1,generator-5238-1,generator-6244-1
,Float64,Float64,Float64,Float64,Float64
1,1.0,1.0,1.0,1.0,1.0
2,1.0,1.0,1.0,1.0,1.0
3,1.0,1.0,1.0,1.0,1.0
4,1.0,1.0,1.0,1.0,1.0
5,1.0,1.0,1.0,1.0,1.0
6,1.0,1.0,1.0,1.0,1.0
7,1.0,1.0,1.0,1.0,1.0
8,1.0,1.0,1.0,1.0,1.0
9,1.0,1.0,1.0,1.0,1.0


---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*